In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance

In [4]:
##LECTURA DEL FICHERO SPSS Y SUS METADATOS
##Debe estar instalado el paquete de pyreadstat que permite leer ficheros SPSS y SAS
import pyreadstat

df, meta = pyreadstat.pyreadstat.read_sav('D:\TFM\Cuestionarios\CuestionariosEsp\Estudiantes\estudiantesESP1.sav', metadataonly=True)
df2, meta2 = pyreadstat.pyreadstat.read_sav('D:\TFM\Cuestionarios\CuestionariosEsp\Escuelas\Limpieza\escuelasESP1.sav')
df3, meta3 = pyreadstat.pyreadstat.read_sav('D:\TFM\Cuestionarios\CuestionariosEsp\profesores\Limpieza\profesoresESP1.sav', metadataonly=True)

In [5]:
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 999
pd.options.display.max_columns = 9999

In [6]:
data = pd.read_csv('EstuTranTar.csv')

In [7]:
df2 = df2[['SC013Q01TA','CNTSCHID']]

In [8]:
data = pd.merge(data, df2,on='CNTSCHID')

In [9]:
#Eliminamos los valores nulos
data = data.dropna()

In [11]:
#Se borran las columnas que no van a entrar al modelo
## Tambien Eliminamos la altura y el peso, ya que da demasiada información
dataMod = data.drop(['Unnamed: 0','CNTSCHID','CNTSTUID','LECTURA1','MATES1','CIENCIA1','GLOBAL1','LECTURA2','MATES2','CIENCIA2','GLOBAL2','ST004D01T','WB151Q01HA','WB152Q01HA','ST003D02T','SC013Q01TA'],axis=1)

## Se divide los datos en test y train

In [12]:
# training /test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

std_sc = StandardScaler()
x = std_sc.fit_transform(dataMod.values)
y = data['SC013Q01TA'].values

testsize = 0.2 # in the [0,1] range. 1: 100%
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=testsize, random_state=1)

In [13]:
from sklearn.ensemble import RandomForestClassifier

std_sc = StandardScaler()
x = std_sc.fit_transform(dataMod.values)
#x = dataMod.values



lr_m =  RandomForestClassifier(n_estimators=400, random_state=50)

#ajuste sobre la muestra completa, sin split train-test
lr_m.fit(X_train, y_train)



RandomForestClassifier(n_estimators=400, random_state=50)

## Validamos el modelo con los datos de test

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred = np.clip(lr_m.predict(X_test), 0,  100.)



In [15]:
mat_confusion = confusion_matrix(
                    y_true    = y_test,
                    y_pred    = y_pred
                )

accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    =y_pred,
            normalize = True
           )

print("Matriz de confusión")
print("-------------------")
print(mat_confusion)
print("")
print(f"El accuracy de test es: {100 * accuracy} %")
    

Matriz de confusión
-------------------
[[4050  332]
 [1633  968]]

El accuracy de test es: 71.86023199198053 %


In [16]:
#Dataframe con el coeficiente y nombre columna
VarCoef = pd.DataFrame(lr_m.feature_importances_.T,dataMod.columns.to_list()).sort_values(0)
VarCoef ['Cols']=VarCoef.index

#Dataframe con las descripciones (convertimos un diccionario a un DataFrame)
Des1 = pd.DataFrame([[key, meta.column_names_to_labels[key]] for key in meta.column_names_to_labels.keys()], columns=['Cols', 'Desc'])
Des2 = pd.DataFrame([[key, meta2.column_names_to_labels[key]] for key in meta2.column_names_to_labels.keys()], columns=['Cols', 'Desc'])
Des3 = pd.DataFrame([[key, meta3.column_names_to_labels[key]] for key in meta3.column_names_to_labels.keys()], columns=['Cols', 'Desc'])
Des = pd.concat([Des1,Des2, Des3],axis=0)


#Hacemos todos los cruces, para que crucen todas las columnas
VarCoef0 = pd.merge(left=Des,right=VarCoef, left_on='Cols', right_on='Cols')

VarCoef0["Coefi_abs"] = abs(VarCoef0[0])
VarImp = VarCoef0.sort_values("Coefi_abs", ascending=False)
VarImp

,Cols,Desc,0,Coefi_abs
32,TMINS,Learning time (minutes per week) - in total,0.003740,0.003740
29,MMINS,Learning time (minutes per week) - <Mathematics>,0.003481,0.003481
31,SMINS,Learning time (minutes per week) - <science>,0.003023,0.003023
30,LMINS,Learning time (minutes per week) - <test language>,0.002930,0.002930
22,ST059Q04HA,Typically required to attend: Number of <class periods> per week in foreign language,0.002732,0.002732
23,ST060Q01NA,"In a normal, full week at school, how many <class periods> are you required to attend in total?",0.002643,0.002643
21,ST059Q03TA,Typically required to attend: Number of <class periods> per week in <science>,0.002294,0.002294
24,ST061Q01NA,"How many minutes, on average, are there in a <class period>?",0.002271,0.002271
27,EC159Q01HA,"On the most recent day you attended school, how long did you study after leaving school? Hours",0.001747,0.001747
28,EC159Q02HA,"On the most recent day you attended school, how long did you study after leaving school? Minutes",0.001601,0.001601


NameError: name 'Ridge' is not defined